# Bekerja dengan Data

Data adalah fondasi tempat model pembelajaran mesin dibangun. Mengelola data secara terpusat di cloud, dan membuat data dapat diakses oleh tim ilmuwan data yang menjalankan eksperimen dan model pelatihan di beberapa stasiun kerja dan target komputasi adalah bagian penting dari solusi ilmu data profesional.

Di buku catatan ini, Anda akan menjelajahi dua objek Azure Machine Learning untuk bekerja dengan data: *penyimpanan data*, dan *himpunan data*.

## Menghubungkan ke ruang kerja Anda

Untuk memulai, hubungkan ke ruang kerja Anda.

> **Catatan**: Jika Anda belum membuat sesi yang terautentikasi dengan langganan Azure, Anda akan diminta untuk mengautentikasi dengan mengklik tautan, memasukkan kode autentikasi, dan masuk ke Azure.

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## Bekerja dengan penyimpanan data

Di Azure Machine Learning, *penyimpanan data* adalah referensi ke lokasi penyimpanan, seperti kontainer blob Azure Storage. Setiap ruang kerja memiliki penyimpanan data default - biasanya kontainer blob penyimpanan Azure yang dibuat dengan ruang kerja. Jika Anda perlu bekerja dengan data yang disimpan di lokasi yang berbeda, Anda dapat menambahkan penyimpanan data kustom ke ruang kerja Anda dan mengatur penyimpanan data kustom ke default.

### Melihat penyimpanan data

Jalankan kode berikut untuk menentukan penyimpanan data di ruang kerja Anda:

In [ ]:
# Get the default datastore
default_ds = ws.get_default_datastore()

# Enumerate all datastores, indicating which is the default
for ds_name in ws.datastores:
    print(ds_name, "- Default =", ds_name == default_ds.name)

Anda juga dapat melihat dan mengelola penyimpanan data di ruang kerja Anda di halaman **Penyimpanan Data** untuk ruang kerja Anda di [Studio Azure Machine Learning](https://ml.azure.com).

### Mengunggah data ke penyimpanan data

Sekarang setelah Anda menentukan penyimpanan data yang tersedia, Anda dapat mengunggah file dari sistem file lokal ke penyimpanan data sehingga dapat diakses oleh eksperimen yang berjalan di ruang kerja, di mana pun skrip eksperimen sedang dijalankan.

In [ ]:
from azureml.core import Dataset
from azureml.data.datapath import DataPath

Dataset.File.upload_directory(src_dir='data',
                              target=DataPath(default_ds, 'diabetes-data/')
                              )

## Bekerja dengan himpunan data

Azure Machine Learning menyediakan abstraksi untuk data dalam bentuk *himpunan data*. Himpunan data adalah referensi yang memiliki versi ke himpunan data tertentu yang mungkin ingin Anda gunakan dalam eksperimen. Himpunan data dapat berbasis *tabular* atau *file*.

### Membuat himpunan data tabular

Mari kita buat himpunan data dari data diabetes yang Anda unggah ke penyimpanan data, dan lihat 20 catatan pertama. Dalam kasus ini, data dalam format terstruktur dalam file CSV, jadi kita akan menggunakan himpunan data *tabular*.

In [ ]:
from azureml.core import Dataset

# Get the default datastore
default_ds = ws.get_default_datastore()

#Create a tabular dataset from the path on the datastore (this may take a short while)
tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

# Display the first 20 rows as a Pandas dataframe
tab_data_set.take(20).to_pandas_dataframe()

Seperti yang dapat Anda lihat pada kode di atas, mudah untuk mengonversi himpunan data tabular ke kerangka data Pandas, memungkinkan Anda bekerja dengan data menggunakan teknik python umum.

### Membuat Himpunan Data file

Himpunan data yang Anda buat adalah himpunan data *tabular* yang dapat dibaca sebagai kerangka data yang berisi semua data dalam file terstruktur yang disertakan dalam definisi himpunan data. Ini berfungsi dengan baik untuk data tabular, tetapi dalam beberapa skenario pembelajaran mesin, Anda mungkin perlu bekerja dengan data yang tidak terstruktur; atau Anda mungkin hanya ingin menangani pembacaan data dari file dalam kode Anda sendiri. Untuk mencapai hal ini, Anda dapat menggunakan himpunan data *file*, yang mencantumkan jalur file di titik pemasangan virtual, yang dapat Anda gunakan untuk membaca data dalam file.

In [ ]:
#Create a file dataset from the path on the datastore (this may take a short while)
file_data_set = Dataset.File.from_files(path=(default_ds, 'diabetes-data/*.csv'))

# Get the files in the dataset
for file_path in file_data_set.to_path():
    print(file_path)

### Mendaftarkan himpunan data

Sekarang setelah Anda membuat himpunan data yang mereferensikan data diabetes, Anda dapat mendaftarkannya agar mudah diakses oleh eksperimen apa pun yang dijalankan di ruang kerja.

Kita akan mendaftarkan himpunan data tabular sebagai **himpunan data diabetes**, dan himpunan data file sebagai **file diabetes**.

In [ ]:
# Register the tabular dataset
try:
    tab_data_set = tab_data_set.register(workspace=ws, 
                                        name='diabetes dataset',
                                        description='diabetes data',
                                        tags = {'format':'CSV'},
                                        create_new_version=True)
except Exception as ex:
    print(ex)

# Register the file dataset
try:
    file_data_set = file_data_set.register(workspace=ws,
                                            name='diabetes file dataset',
                                            description='diabetes files',
                                            tags = {'format':'CSV'},
                                            create_new_version=True)
except Exception as ex:
    print(ex)

print('Datasets registered')

Anda dapat melihat dan mengelola himpunan data di halaman **Himpunan Data** untuk ruang kerja Anda di [Studio Azure Machine Learning](https://ml.azure.com). Anda juga bisa mendapatkan daftar himpunan data dari objek ruang kerja:

In [ ]:
print("Datasets:")
for dataset_name in list(ws.datasets.keys()):
    dataset = Dataset.get_by_name(ws, dataset_name)
    print("\t", dataset.name, 'version', dataset.version)

Kemampuan untuk menetapkan versi himpunan data memungkinkan Anda mendefinisikan ulang himpunan data tanpa merusak eksperimen atau alur yang ada yang mengandalkan definisi sebelumnya. Secara default, versi terbaru dari himpunan data bernama dikembalikan, tetapi Anda dapat mengambil versi tertentu dari himpunan data dengan menentukan nomor versi, seperti ini:

```python
dataset_v1 = Dataset.get_by_name(ws, 'diabetes dataset', version = 1)
```


### Melatih model dari himpunan data tabular

Sekarang setelah Anda memiliki himpunan data, Anda siap untuk memulai model pelatihan dari himpunan data. Anda dapat meneruskan himpunan data ke skrip sebagai *input* dalam penghitung yang digunakan untuk menjalankan skrip.

Jalankan dua sel kode berikut untuk membuat:

1. Folder bernama **diabetes_training_from_tab_dataset**
2. Skrip yang melatih model klasifikasi dengan menggunakan himpunan data tabular yang diteruskan sebagai argumen.

In [ ]:
import os

# Create a folder for the experiment files
experiment_folder = 'diabetes_training_from_tab_dataset'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# Import libraries
import os
import argparse
from azureml.core import Run, Dataset
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Get the script arguments (regularization rate and training dataset ID)
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
parser.add_argument("--input-data", type=str, dest='training_dataset_id', help='training dataset')
args = parser.parse_args()

# Set regularization hyperparameter (passed as an argument to the script)
reg = args.reg_rate

# Get the experiment run context
run = Run.get_context()

# Get the training dataset
print("Loading Data...")
diabetes = run.input_datasets['training_data'].to_pandas_dataframe()

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

> **Catatan**: Dalam skrip, himpunan data diteruskan sebagai parameter (atau argumen). Dalam kasus himpunan data tabular, argumen ini akan berisi ID dari himpunan data terdaftar; jadi Anda dapat menulis kode dalam skrip untuk mendapatkan ruang kerja eksperimen dari konteks eksekusi, lalu mendapatkan himpunan data menggunakan ID himpunan data; seperti ini:
>
> ```
> run = Run.get_context()
> ws = run.experiment.workspace
> dataset = Dataset.get_by_id(ws, id=args.training_dataset_id)
> diabetes = dataset.to_pandas_dataframe()
> ```
>
> Tetapi, eksekusi Azure Machine Learning mengidentifikasi argumen yang mereferensikan himpunan data bernama secara otomatis dan menambahkannya ke kumpulan **input_datasets** eksekusi, sehingga Anda juga dapat mengambil himpunan data dari koleksi ini dengan menentukan "nama ramah" koleksi (yang akan Anda lihat sebentar lagi, ditentukan dalam definisi argumen dalam konfigurasi eksekusi skrip untuk eksperimen). Ini merupakan pendekatan yang diambil dalam skrip di atas.

Sekarang Anda dapat menjalankan skrip sebagai eksperimen, menentukan argumen untuk himpunan data pelatihan, yang dibaca oleh skrip.

> **Catatan**: Kelas **Himpunan Data** bergantung pada beberapa komponen dalam paket **azureml-dataprep**, jadi Anda harus menyertakan paket ini di lingkungan tempat eksperimen pelatihan akan dijalankan. Paket **azureml-dataprep** disertakan dalam paket **azure-defaults**.

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.runconfig import DockerConfiguration
from azureml.widgets import RunDetails


# Create a Python environment for the experiment (from a .yml file)
env = Environment.from_conda_specification("experiment_env", "environment.yml")

# Get the training dataset
diabetes_ds = ws.datasets.get("diabetes dataset")

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                              script='diabetes_training.py',
                              arguments = ['--regularization', 0.1, # Regularizaton rate parameter
                                           '--input-data', diabetes_ds.as_named_input('training_data')], # Reference to dataset
                              environment=env,
                              docker_runtime_config=DockerConfiguration(use_docker=True)) 

# submit the experiment
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

> **Catatan:** Argumen **--input-data** meneruskan himpunan data sebagai *input bernama* yang menyertakan *nama ramah* untuk himpunan data, yang digunakan oleh skrip untuk membacanya dari kumpulan **input_datasets** dalam eksperimen yang dijalankan. Nilai string dalam argumen **--input-data** sebenarnya adalah ID himpunan data yang terdaftar.  Sebagai pendekatan alternatif, Anda cukup meneruskan `diabetes_ds.id`, dalam kasus ini skrip dapat mengakses ID himpunan data dari argumen skrip dan menggunakannya untuk mendapatkan himpunan data dari ruang kerja, tetapi tidak dari kumpulan **input_datasets**.

Pertama kali eksperimen dijalankan, mungkin diperlukan beberapa waktu untuk menyiapkan lingkungan Python - eksekusi selanjutnya akan lebih cepat.

Setelah eksperimen selesai, di widget, lihat log output **azureml-logs/70_driver_log.txt** dan metrik yang dihasilkan oleh eksekusi.

### Mendaftarkan model terlatih

Seperti eksperimen pelatihan apa pun, Anda dapat mengambil model terlatih dan mendaftarkan model di ruang kerja Azure Machine Learning Anda.

In [ ]:
from azureml.core import Model

run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Tabular dataset'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

### Melatih model dari himpunan data file

Anda telah melihat cara melatih model menggunakan data pelatihan dalam himpunan data *tabular*; tetapi bagaimana dengan himpunan data *file*?

Saat Anda menggunakan himpunan data file, argumen himpunan data yang diteruskan ke skrip mewakili titik pemasangan yang berisi jalur file. Cara Anda membaca data dari file ini bergantung pada jenis data dalam file dan apa yang ingin Anda lakukan dengan file. Dalam kasus file CSV diabetes, Anda dapat menggunakan modul **glob** Python untuk membuat daftar file di titik pemasangan virtual yang ditentukan oleh himpunan data, dan membaca semuanya ke dalam kerangka data Pandas yang digabungkan menjadi kerangka data tunggal.

Jalankan dua sel kode berikut untuk membuat:

1. Folder bernama **diabetes_training_from_file_dataset**
2. Skrip yang melatih model klasifikasi dengan menggunakan himpunan data file yang diteruskan adalah sebagai *input*.

In [ ]:
import os

# Create a folder for the experiment files
experiment_folder = 'diabetes_training_from_file_dataset'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# Import libraries
import os
import argparse
from azureml.core import Dataset, Run
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import glob

# Get script arguments (rgularization rate and file dataset mount point)
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
parser.add_argument('--input-data', type=str, dest='dataset_folder', help='data mount point')
args = parser.parse_args()

# Set regularization hyperparameter (passed as an argument to the script)
reg = args.reg_rate

# Get the experiment run context
run = Run.get_context()

# load the diabetes dataset
print("Loading Data...")
data_path = run.input_datasets['training_files'] # Get the training data path from the input
# (You could also just use args.dataset_folder if you don't want to rely on a hard-coded friendly name)

# Read the files
all_files = glob.glob(data_path + "/*.csv")
diabetes = pd.concat((pd.read_csv(f) for f in all_files), sort=False)

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

Sama seperti himpunan data tabular, Anda dapat mengambil himpunan data file dari kumpulan **input_datasets** dengan menggunakan nama ramah himpunan data file. Anda juga dapat mengambil himpunan data file dari argumen skrip, yang dalam kasus himpunan data file berisi jalur pemasangan ke file (bukan ID himpunan data yang diteruskan untuk himpunan data tabular).

Selanjutnya kita perlu mengubah cara kita meneruskan himpunan data ke skrip - kita perlu menentukan jalur tempat skrip dapat membaca file. Anda dapat menggunakan metode **as_download** atau **as_mount** untuk melakukan hal ini. Dengan menggunakan **as_download** akan menyebabkan file dalam himpunan data file diunduh ke lokasi sementara pada komputasi tempat skrip dijalankan, sementara **as_mount** membuat titik pemasangan tempat file dapat dialirkan langsung dari penyimpanan data.

Anda dapat menggabungkan metode akses dengan metode **as_named_input** untuk menyertakan himpunan data dalam kumpulan **input_datasets** dalam eksekusi eksperimen (jika Anda menghilangkannya, misalnya dengan mengatur argumen ke `diabetes_ds.as_mount()`, skrip akan dapat mengakses titik pemasangan himpunan data dari argumen skrip, tetapi tidak dari kumpulan **input_datasets**).

In [ ]:
from azureml.core import Experiment
from azureml.core.runconfig import DockerConfiguration
from azureml.widgets import RunDetails


# Get the training dataset
diabetes_ds = ws.datasets.get("diabetes file dataset")

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='diabetes_training.py',
                                arguments = ['--regularization', 0.1, # Regularizaton rate parameter
                                             '--input-data', diabetes_ds.as_named_input('training_files').as_download()], # Reference to dataset location
                                environment=env, # Use the environment created previously
                                docker_runtime_config=DockerConfiguration(use_docker=True))

# submit the experiment
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

Setelah eksperimen selesai, di widget, lihat log output **azureml-logs/70_driver_log.txt** untuk memverifikasi bahwa file dalam himpunan data file telah diunduh ke folder sementara guna mengizinkan skrip untuk membaca file.

### Mendaftarkan model terlatih

Sekali lagi, Anda dapat mendaftarkan model yang dilatih oleh eksperimen.

In [ ]:
from azureml.core import Model

run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'File dataset'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **Informasi Selengkapnya**: Untuk informasi selengkapnya tentang pelatihan dengan himpunan data, lihat [Pelatihan dengan Himpunan Data](https://docs.microsoft.com/azure/machine-learning/how-to-train-with-datasets) di dokumentasi Azure Machine Learning.